# Generative Adversarial Networks (GANs) for Image Generation

Generative Adversarial Networks (GANs) consist of two neural networks, the generator and the discriminator, which compete against each other in a game-theoretic scenario.

* Generator: Generates fake data (images in this case) from random noise.
* Discriminator: Tries to distinguish between real data (real images) and fake data (generated images).

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Load dataset (e.g., CIFAR-10)
(train_images, _), (_, _) = tf.keras.datasets.cifar10.load_data()
train_images = train_images / 255.0

# Generator model
def build_generator():
    model = models.Sequential([
        layers.Dense(256, input_shape=(100,)),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 4)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(3, (3, 3), padding='same', activation='tanh')
    ])
    return model

# Discriminator model
def build_discriminator():
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), input_shape=(32, 32, 3), padding='same'),
        layers.LeakyReLU(),
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.LeakyReLU(),
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

generator = build_generator()
discriminator = build_discriminator()

# Compile models
discriminator.compile(optimizer='adam', loss='binary_crossentropy')
discriminator.trainable = False

gan_input = layers.Input(shape=(100,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = models.Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Train GAN
batch_size = 64
for epoch in range(10000):
    # Train discriminator
    idx = np.random.randint(0, train_images.shape[0], batch_size)
    real_images = train_images[idx]
    noise = np.random.normal(0, 1, (batch_size, 100))
    generated_images = generator.predict(noise)
    discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
    discriminator.train_on_batch(generated_images, np.zeros((batch_size, 1)))
    
    # Train generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    gan.train_on_batch(noise, np.ones((batch_size, 1)))

# Generate images
noise = np.random.normal(0, 1, (10, 100))
generated_images = generator.predict(noise)
for img in generated_images:
    plt.imshow(img)
    plt.show()


## Training Process:

1. The generator produces a batch of images from random noise.
2. The discriminator evaluates this batch alongside a batch of real images and tries to differentiate between real and fake images.
3. The generator tries to fool the discriminator by producing more realistic images over time.
4. The loss for the generator is based on how well it can fool the discriminator.
5. The loss for the discriminator is based on its ability to correctly classify real and fake images.